# 使用2-sat算法实现扫雷

大家好!，最近写了一个可以实现推理能力的扫雷程序。通过图论的方法可以实现扫雷的逻辑推理。

该项目是通过`2-SAT`算法和`tarjan`算法，推导和求解扫雷游戏中的某些结构。在扫雷游戏中，我们通过分析已知方块状态和邻接方块的提示信息，推理出哪些方块是雷，哪些不是雷。

<table>
    <tr>
        <td>
            <img src="./assert/image.png" alt="图片1" style="width: 500px; height: auto;">
        </td>
        <td>
            <img src="./assert/image2.png" alt="图片2" style="width: 500px; height: auto;">
        </td>
    </tr>
</table>

## 扫雷💣游戏规则

网格中存在许多方块，其中随机分布着一些雷。你的目标是避开这些雷，打开所有空白格子。一个非雷格子中的数字表示其相邻8格中的雷数。

你可以利用这些信息推导出安全格子和雷的位置，你可以用鼠标右键标注你认为是雷的位置，用鼠标左键打开安全的位置。


**程序中各种颜色的释义：**

- 红：用户标注的雷的位置
- 绿：计算机推导出认为是非雷的位置
- 蓝：计算机推导出认为是雷的位置。

## 2-SAT 算法

2-SAT问题是一种布尔逻辑问题，目标是判断一个包含若干个布尔变量的布尔表达式是否满足。每个子表达式包含两个变量及其取值。对应扫雷问题就是“每个方块要么有雷，要么没有雷”。

如，洛谷[P4782题](https://www.luogu.com.cn/problem/P4782)：

<img src="./assert/image3.png" alt="图片3" style="width: 500px; height: auto;">

通常，通过构建一个图，并利用`强连通分量`(SCC)来解决2-SAT问题。

**2-SAT 问题解决步骤**

1. **构造蕴含关系**：

    根据布尔表达式的约束条件，构建一个有向图。如有布尔逻辑如下：a V b
    
    可以将`或关系`转为`蕴含关系`：

    $ a \vee b \Leftrightarrow (\neg a \rightarrow b) \wedge (\neg b \rightarrow a)$
    **解释为：** 如果a不成立，那么b一定成立，且 如果b不成立，那么a一定成立。

2. **建图**

    把每个变量看成点，由于n个点都存在两种状态，因为需要2n个节点。其中前n个节点表示这些节点不成立，后n个节点表示这些节点成立。

    即：把 $x_i$ 拆为 节点i和节点n+i，其中 节点i表示 $x_i = false$， 节点n+i表示 $x_i=true$

    蕴含关系就表示为有向边，例如  $\neg b \rightarrow a$ 存在一条从`节点1` 到 `节点 0+n`的有向边。

3. **可行性判断**

    `强连通分量SCC：` 图中的一组节点，其中每两个节点可以互相访问。

    （1）: 如果某个`变量`与其`反变量`在同一个强连通分量中，即会存在如下情况：

    $ (\neg A \rightarrow A) \wedge (A \rightarrow \neg A )$

    则说明，如果A是雷，那么A就不是雷，如果A不是雷，那么A就是雷。 这种情况是不会存在的，所以这种状态无解。

    （2）: 如果不在同一个强连通分量，则说明两个变量要么是在链⛓️上，要么就是两个节点并不连通。

    如果两个节点并不连通，也是无解。

    如果在链⛓️上，则存在两种状态：

    $(\neg A \rightarrow A)$  ：A不是雷的话，那么A就是雷，那么就和A不是雷的条件矛盾了。说明A是雷。

    $(A \rightarrow \neg A )$ ：A是雷的话，那么A就不是雷，那么就和A是雷的条件矛盾了。说明A不是雷。


## Tarjan算法

Tarjan算法是一种用于寻找有向图强连通分量(SCC)的经典算法。Tarjan算法通过使用深度优先搜索(DFS) 递归地访问每一个节点，计算节点的`低链接值`来识别强连通分量。

### Tarjan算法步骤
1: **时间戳dfn[x]**: 节点x被第一次访问的顺序
2: **低链接值low[x]**: 从节点x出发，能访问到的最早时间戳。

1. 对每个节点进行深度优先搜索，访问时，将节点设置时间戳，将节点入栈

2. 枚举x的邻居节点y，分三种情况
    1) y尚未访问： 深搜y，返回时，对比用若low[y]比low[x]更小，则更新low[x]
    2) 若y已经访问，且在栈中，则用dfn[y]和low[x]比较，若更小，则更新low[x]
    3) 若y已经访问，且不在栈中，则pass

3. 离开节点x时，若dfn[x] == low[x]， 则找到了一个scc，while 直到x出栈。记录scc。



## 使用tarjan算法解决2-sat问题

建图之后，通过tarjan算法计算scc

1: 若节点i和节点i+n的scc记录值相同，说明两者在同一强连通分量中，则 $x_i$ 无解。

2: 若scc记录值不同，由于tarjan算法是栈结构，scc记录值小的值说明后出栈。

1. 通过深度优先搜索判断节点i和节点i+n是否连通，若连通：

    1) 若scc[i] < scc[i+n] 则说明节点i后出栈，则存在
        $\neg x_i \rightarrow x_i$, 说明 $x_i$ 有雷

    2) 若scc[i] > scc[i+n] 则说明节点i后出栈，则存在
        $x_i \rightarrow \neg x_i$, 说明 $x_i$ 无雷

2. 若不连通，则无解


### 代码模版
```cpp
#include <iostream>
#include <vector>
#include <stack>

using namespace std;

const int MAXN = 2000005; // 2 * 10^6

vector<int> graph[MAXN];
int dfn[MAXN], low[MAXN], scc[MAXN], instack[MAXN];
stack<int> st;
int timestamp = 0, cnt = 0;

void tarjan(int x) {
    dfn[x] = low[x] = ++timestamp;
    st.push(x);
    instack[x] = 1;
    
    for (int neighbor : graph[x]) {
        if (!dfn[neighbor]) {
            tarjan(neighbor);
            low[x] = min(low[x], low[neighbor]);
        } else if (instack[neighbor]) {
            low[x] = min(low[x], dfn[neighbor]);
        }
    }
    
    if (dfn[x] == low[x]) {
        ++cnt;
        while (true) {
            int y = st.top();
            st.pop();
            instack[y] = 0;
            scc[y] = cnt;
            if (y == x) break;
        }
    }
}

int main() {
    ios::sync_with_stdio(false);
    cin.tie(nullptr);
    
    int n, m;
    cin >> n >> m;
    
    for (int i = 0; i < m; i++) {
        int a, state_a, b, state_b;
        cin >> a >> state_a >> b >> state_b;
        --a, --b;
        
        graph[a + n * state_a].push_back(b + n * (1 - state_b));
        graph[b + n * state_b].push_back(a + n * (1 - state_a));
    }
    
    for (int i = 0; i < 2 * n; i++) {
        if (!dfn[i]) tarjan(i);
    }
    
    for (int i = 0; i < n; i++) {
        if (scc[i] == scc[i + n]) {
            cout << "IMPOSSIBLE\n";
            return 0;
        }
    }
    
    cout << "POSSIBLE\n";
    for (int i = 0; i < n; i++) {
        cout << (scc[i] < scc[i + n] ? 1 : 0) << " ";
    }
    cout << "\n";
    
    return 0;
}

```
    

## 举例说明

A B C D

X 1 2 X

存在以上扫雷场景，其中X表示非雷网格，A B C D四个未知网格。

### 1 表示 A B C中存在一个雷,可以构造以下蕴含关系：其中A表示A是雷，非A表示A不是雷

$A \rightarrow \neg B$

$A \rightarrow \neg C$

$B \rightarrow \neg A$

$B \rightarrow \neg C$

$C \rightarrow \neg A$

$C \rightarrow \neg B$

### 2 表示B C D中存在一个雷,可以构造以下蕴含关系

$\neg B \rightarrow C$

$\neg B \rightarrow D$

$\neg C \rightarrow B$

$\neg C \rightarrow D$

$\neg D \rightarrow B$

$\neg D \rightarrow C$


计算可得 A为0， B为-1， C为-1，D为1 其中0表示无雷，-1表示无解，1表示有雷。

通过以上算法和建图规则，即可实现扫雷的推导🎉。

## 实现细节

建图的代码：

```javascript
if (this.cells[item].numBombAroud - flag_num === 1) {
    for (let guess_true_cell of blank_list) {
        for (let other_cell of blank_list) {
            if (guess_true_cell === other_cell)
                continue
            this.graph[guess_true_cell + total_length].push(other_cell);
        }
    }
}
if (blank_num - this.cells[item].numBombAroud + flag_num === 1) {
    for (let guess_false_cell of blank_list) {
        for (let other_cell of blank_list) {
            if (guess_false_cell === other_cell)
                continue
            this.graph[guess_false_cell].push(other_cell + total_length);
        }
    }
}
```

## 展望与优化

目前建图的时候只是考虑了多个方块有一个雷，以及多个方块只有一个不是雷。

其实还可以考虑2个雷，或者2个不是雷的情况，甚至3个及以上。只要多一个for循环即可。

## 链接：

游戏链接：https://zhenghong-liu.github.io/Label-Galaxy/dist/index.html#/bombs

代码链接：https://github.com/Zhenghong-Liu/Label-Galaxy/blob/master/src/components/page/Bombs.vue

## 参考

1. https://www.cnblogs.com/ambition/archive/2011/09/22/Mine-sweeping.html

2. https://www.bilibili.com/video/BV1kE421w7XR/?spm_id_from=333.1007.top_right_bar_window_custom_collection.content.click&vd_source=099edd8ba094e7ddc14190b99d15a5fe

3. https://www.bilibili.com/video/BV1SY411M7Tv/?spm_id_from=333.1007.top_right_bar_window_custom_collection.content.click&vd_source=099edd8ba094e7ddc14190b99d15a5fe

4. https://www.luogu.com.cn/problem/P4782